<a id=top></a>

# **目次**

<b>
    <details>
        <summary>
            <a href="#modules", style="font-size: xx-large">1. モジュールインポート</a>
        </summary>
    </details>
    <details>
        <summary>
            <a href="#data", style="font-size: xx-large">2. データインポート</a>
        </summary>
    </details>
    <details>
        <summary>
            <a href="#meltbulk", style="font-size: xx-large">3. バルクデータ</a>
        </summary>
        <table></table>
    </details>
    <details>
        <summary>
            <a href="#output", style="font-size: xx-large">4. ファイルに出力</a>
        </summary>
    </details>
</b>


In [ ]:
import pandas as pd
import numpy as np
from glob import glob

import datetime
import pytz


In [ ]:
global data_dir, file_name_list
data_dir = '../../data/interim/internal/'
output_dir = '../../data/interim/internal/merged/'
file_name_list = [
                  'hr', # 特許権者
                  'ipc', # IPC
                  'reg_info' # 登録情報
                  ]


In [ ]:
bulk_stack_df_dict = {
                      file_name: pd.concat([pd.read_csv(f'{data_dir}/{bs}/{file_name}.csv', 
                                                        sep=',', 
                                                        encoding='utf-8', 
                                                        dtype=str) for bs in ['bulk', 'stack']], 
                                           ignore_index=True, 
                                           axis='index')
                      for file_name in file_name_list
                      }


In [ ]:
df = pd.merge(bulk_stack_df_dict['reg_info'].copy(), bulk_stack_df_dict['hr'].copy(), 
              on='reg_num', 
              how='inner')

df = pd.merge(df, bulk_stack_df_dict['ipc'].copy(), 
              on='app_num', 
              how='inner')

df = df.drop(columns=['app_num'])\
       .drop_duplicates(keep='first')\
       .reset_index(drop=True)


In [ ]:
jst = pytz.timezone('Asia/Tokyo')
now = datetime.datetime.now(jst)
str_now = now.strftime('%Y%m')

df.to_csv(f'{output_dir}{str_now}.csv', 
          sep=',', 
          encoding='utf-8', 
          index=False)
